In [10]:
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import undetected_chromedriver as uc
from fake_useragent import UserAgent
from tqdm.notebook import tqdm
from selenium.webdriver.chrome.options import Options


from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime


In [21]:
from datetime import datetime, timedelta

# Función para generar combinaciones de fechas de ida y regreso
def generar_combinaciones(fecha_inicial, duracion, fechas_disponibles):
    combinaciones = []
    for i in range(len(fechas_disponibles) - duracion + 1):
        fecha_ida = fechas_disponibles[i]
        fecha_regreso = fechas_disponibles[i + duracion - 1]
        combinaciones.append((fecha_ida, fecha_regreso))
    return combinaciones

# Función para generar enlaces con fechas específicas
def generar_enlace(fecha_ida, fecha_regreso):
    url_base = 'https://www.bestday.com.mx/shop/flights/results/roundtrip/MEX/MIA/{}/{}/1/0/0?from=SB&di=1-0&reSearch=true#showModal'
    return url_base.format(fecha_ida.strftime('%Y-%m-%d'), fecha_regreso.strftime('%Y-%m-%d'))

while True:
    try:
        # Obtener la duración del viaje y las fechas disponibles desde el usuario
        duracion_del_viaje = int(input("Ingrese la duración del viaje en días: "))
        fecha_inicio_str = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
        fecha_fin_str = input("Ingrese la fecha de fin (YYYY-MM-DD): ")

        # Convertir las cadenas de fecha a objetos datetime
        fecha_inicio = datetime.strptime(fecha_inicio_str, '%Y-%m-%d')
        fecha_fin = datetime.strptime(fecha_fin_str, '%Y-%m-%d')

        # Generar la lista de fechas disponibles entre la fecha de inicio y fin
        fechas_disponibles = [fecha_inicio + timedelta(days=i) for i in range((fecha_fin - fecha_inicio).days + 1)]

        # Verificar que la duración del viaje sea menor o igual al rango de fechas
        if duracion_del_viaje > len(fechas_disponibles):
            raise ValueError("La duración del viaje es mayor al rango de fechas disponibles.")

        # Generar combinaciones de fechas de ida y regreso
        combinaciones = generar_combinaciones(fecha_inicio, duracion_del_viaje, fechas_disponibles)

        # Generar enlaces para cada combinación
        enlaces_generados = [generar_enlace(ida, regreso) for ida, regreso in combinaciones]

        # Imprimir los enlaces generados
        print("Enlaces generados:")
        for enlace in enlaces_generados:
            print(enlace)

        # Si todo es exitoso, salimos del bucle
        break

    except ValueError as e:
        print(f"Error: {e}. Asegúrate de ingresar fechas en el formato correcto (YYYY-MM-DD). Intenta nuevamente.")



In [16]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


options = uc.ChromeOptions() 
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
options.headless = False 
driver = uc.Chrome(options=options)
driver.maximize_window()
driver.set_page_load_timeout(120)
registros=[]
registros2=[]
registros3=[]

#BESTDAY ROUND TRIP SRAPEO
url='https://www.bestday.com.mx/shop/flights/results/roundtrip/MEX/MIA/2023-12-20/2023-12-31/1/0/0?from=SB&di=1-0&reSearch=true#showModal'
driver.get(url)
time.sleep(5)


# Realizar desplazamiento hacia abajo utilizando JavaScript
for _ in range(4):  # Número de veces que se desplazará hacia abajo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)  # Esperar brevemente entre cada desplazamiento


#viaje REDONDO
sopa=BeautifulSoup(driver.page_source,'html.parser')
#encontrar precios
tarjetas=sopa.find_all('div',{'class':'cluster-pricebox-container CLUSTER'})
for tarjeta in tarjetas:
    precio=tarjeta.find('span',{'class':'amount price-amount'}).text.strip()
    registros.append(precio)


#DESPEGAR.COM ROUND TRIP SCRAPEO
url='https://www.despegar.com.mx/shop/flights/results/roundtrip/MEX/ORL/2023-12-20/2023-12-29/1/0/0?from=SB&di=1-0'
driver.get(url)
time.sleep(5)


# Realizar desplazamiento hacia abajo utilizando JavaScript
for _ in range(4):  # Número de veces que se desplazará hacia abajo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)  # Esperar brevemente entre cada desplazamiento

sopa=BeautifulSoup(driver.page_source,'html.parser')
#encontrar precios 2..0
tarjetas=sopa.find_all('div',{'class':'cluster-pricebox-container CLUSTER'})
for tarjeta in tarjetas:
    precio=tarjeta.find('span',{'class':'amount price-amount'}).text.strip()
    registros2.append(precio)

driver.close()



#KAYAK
url='https://www.kayak.com.mx/flights/MEX-ORL/2024-01-23/2024-01-30?sort=bestflight_a'
driver.get(url)

# Configurar las opciones de Chrome
options = Options()
driver = webdriver.Chrome(options=options)

# Abre la página donde se encuentra el elemento
driver.get("https://www.kayak.com.mx/flights/MEX-ORL/2024-01-23/2024-01-30?sort=bestflight_a")
time.sleep(5)

# Número de clics deseados
num_clics = 3
# Encuentra el elemento por su selector CSS y realiza los clics
for _ in range(num_clics):
    elemento_ver_mas = driver.find_element("css selector", ".ULvh-button.show-more-button")
    # Simula un clic en el elemento
    elemento_ver_mas.click()
    # Espera un momento para ver los resultados (opcional)
    driver.implicitly_wait(20)


#viaje REDONDO
sopa=BeautifulSoup(driver.page_source,'html.parser')
#encontrar precios 3.0
tarjetas=sopa.find_all('div',{'class':'nrc6'})
for tarjeta in tarjetas:
    precio=tarjeta.find('div',{'class':'f8F1-price-text'}).text.strip()
    registros3.append(precio)

driver.close()



TimeoutException: Message: timeout: Timed out receiving message from renderer: 114.278
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00FE6E73+174291]
	(No symbol) [0x00F10AC1]
	(No symbol) [0x00C26FF6]
	(No symbol) [0x00C19B86]
	(No symbol) [0x00C19ABF]
	(No symbol) [0x00C189DC]
	(No symbol) [0x00C18EFD]
	(No symbol) [0x00C222F5]
	(No symbol) [0x00C2F4F4]
	(No symbol) [0x00C32676]
	(No symbol) [0x00C193BB]
	(No symbol) [0x00C2F41C]
	(No symbol) [0x00C8A2D5]
	(No symbol) [0x00C76DA6]
	(No symbol) [0x00C51034]
	(No symbol) [0x00C51F8D]
	GetHandleVerifier [0x010849CC+820268]
	sqlite3_dbdata_init [0x01144EBE+652494]
	sqlite3_dbdata_init [0x011448D9+650985]
	sqlite3_dbdata_init [0x0113962C+605244]
	sqlite3_dbdata_init [0x0114586B+654971]
	(No symbol) [0x00F1FEBC]
	(No symbol) [0x00F18428]
	(No symbol) [0x00F1854D]
	(No symbol) [0x00F05858]
	BaseThreadInitThunk [0x778FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77BC7C6E+286]
	RtlGetAppContainerNamedObjectPath [0

In [15]:
print(registros)

['12,993', '17,998', '12,994', '12,997', '11,481', '12,999', '15,993', '15,997', '15,993', '15,997']


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
options = uc.ChromeOptions() 
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
options.headless = False 
driver = uc.Chrome(options=options)
driver.maximize_window()
driver.set_page_load_timeout(120)
registros=[]
registros3=[]



# Configurar las opciones de Chrome
options = Options()
driver = webdriver.Chrome(options=options)

# Abre la página donde se encuentra el elemento
driver.get("https://www.kayak.com.mx/flights/MEX-ORL/2024-01-23/2024-01-30?sort=bestflight_a")
time.sleep(5)


# Número de clics deseados
num_clics = 3

# Encuentra el elemento por su selector CSS y realiza los clics
for _ in range(num_clics):
    elemento_ver_mas = driver.find_element("css selector", ".ULvh-button.show-more-button")

    # Simula un clic en el elemento
    elemento_ver_mas.click()

    # Espera un momento para ver los resultados (opcional)
    driver.implicitly_wait(20)


#viaje REDONDO
sopa=BeautifulSoup(driver.page_source,'html.parser')

tarjetas=sopa.find_all('div',{'class':'nrc6'})
for tarjeta in tarjetas:
    precio=tarjeta.find('div',{'class':'f8F1-price-text'}).text.strip()
    registros3.append(precio)


In [5]:
print(registros3)

[]


In [ ]:
def encontrar_valores_menores(arreglo1, arreglo2, arreglo3, umbral):
    resultados = []

    # Buscar en el primer arreglo
    for i, valor in enumerate(arreglo1):
        if valor < umbral:
            resultados.append(f"Arreglo 1, Posición {i}")

    # Buscar en el segundo arreglo
    for i, valor in enumerate(arreglo2):
        if valor < umbral:
            resultados.append(f"Arreglo 2, Posición {i}")

    # Buscar en el tercer arreglo
    for i, valor in enumerate(arreglo3):
        if valor < umbral:
            resultados.append(f"Arreglo 3, Posición {i}")

    return resultados

# Ejemplo de uso
arreglo1 = [5, 10, 3, 8, 15]
arreglo2 = [2, 7, 12, 6, 18]
arreglo3 = [9, 4, 11, 14, 1]
umbral = 7000

resultados = encontrar_valores_menores(registros, registros2, registros3, umbral)

for resultado in resultados:
    print(resultado)